In [30]:
# Useful starting lines
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.linear_model as sk
from glove_routines import *
%load_ext autoreload
%autoreload 2

ts_neg = np.load('data/trainingset_neg.npy')
ts_pos = np.load('data/trainingset_pos.npy')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
#Train a Linear Classifier: Train a linear classifier (e.g. logistic regression or SVM) on your constructed 
#features, using the scikit learn library, or your own code from the earlier labs. Recall that the labels 
#indicate if a tweet used to contain a :) or :( smiley.
training_set = np.concatenate((ts_neg,ts_pos))
y = training_set[:,0]
X = training_set[:,1:np.shape(training_set)[1]]
LR = sk.LogisticRegression()
#http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
LR.fit(X,y)
print("End of LR")
#LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, 
#class_weight=None, random_state=None, solver='liblinear', max_iter=100, multi_class='ovr', verbose=0, 
#warm_start=False, n_jobs=1)[source]¶

End of LR


In [53]:
a = LR.coef_
az = LR.predict(X[1:100000])
bz = LR.predict(X[100000:200000])

In [58]:
print(sum(az))
print(sum(bz))
print(np.shape(X))
print(200000/2)

27356.0
40389.0
(200000, 20)
100000.0
